# Data Modeling & Warehouse Concepts: Exercises Results


## 1. Data Modeling Fundamentals
- **a.** Define *data modeling* in your own words. Why is it important in analytics and data engineering?
- **b.** List and briefly describe the three types of data models: *conceptual*, *logical*, and *physical*.

### Answer

- **a.** *Data modeling* is the process of designing and structuring how data is stored, organized, and related within a system. It serves as a blueprint for how information flows and is accessed, ensuring consistency, integrity, and clarity. In analytics and data engineering, data modeling is crucial because it enables efficient querying, reliable reporting, scalable architectures, and easier maintenance, all of which are key for supporting business decisions and data-driven insights.

- **b.** Types of data models:
    - **Conceptual model:** High-level representation that defines the main entities and their relationships, focusing on what data is important without concern for implementation details. Useful for communicating with stakeholders and establishing business requirements.
    - **Logical model:** More detailed than the conceptual model, it defines tables, columns, keys, and relationships in a technology-agnostic way. It translates business requirements into a structured format, preparing for implementation.
    - **Physical model:** The actual implementation of the logical model in a specific database system. It specifies data types, indexes, partitions, and physical storage details, optimizing for performance and constraints of the chosen technology.


---
## 2. Schema Design
- **a.** Given a retail business scenario (sales, products, customers, dates), draw an ER diagram for both a **star schema** and a **snowflake schema**. Label fact and dimension tables.
- **b.** Identify one advantage and one disadvantage of each schema type.



### Answer

**a. ER Diagram for Star and Snowflake Schemas**

- **Star Schema:**

```
               +-------------+
               |  Products   |
               +-------------+
                      |
+-----------+   +-----+-----+   +-------------+   +-------------+
| Customers |---|   Sales   |---|    Dates    |---|   (Facts)    |
+-----------+   +-----------+   +-------------+   +-------------+
   (Dim)           (Fact)           (Dim)             (Dim)
```

- **Snowflake Schema:**

```
               +-------------+
               |  Brands     |
               +-----+-------+
                     |
               +-----v-----+         +-------------+
               | Products  |         | Categories  |
               +-----+-----+         +-------------+
                     |
+-----------+   +-----+-----+   +-------------+   +-------------+
| Customers |---|   Sales   |---|    Dates    |---|   (Facts)    |
+-----+-----+   +-----------+   +-------------+   +-------------+
      |            (Fact)           (Dim)             (Dim)
+-----v-----+                                     
| Addresses |                                    
+-----------+                                    
```
- *In snowflake, dimension tables are further normalized (e.g., products split into brands/categories; customers split into addresses).*

**b. Advantages and Disadvantages**

- **Star Schema**
    - *Advantage:* Simpler design, faster queries for analytics and reporting.
    - *Disadvantage:* Data redundancy in dimension tables (denormalized).

- **Snowflake Schema**
    - *Advantage:* Reduces data redundancy, improves data consistency (more normalized).
    - *Disadvantage:* More complex queries (more joins), harder for end-users to understand.


---
## 3. Fact vs. Dimension Tables
- **a.** For a simple online store, list at least two example fact tables and three dimension tables. Give 2–3 columns for each.
- **b.** Explain the difference between a fact table’s and a dimension table’s typical contents.



### Answer

**a. Example Tables for an Online Store**

- **Fact Tables:**
    1. `orders`
        - Columns: `order_id`, `customer_id`, `order_date`, `total_amount`
    2. `order_items`
        - Columns: `order_item_id`, `order_id`, `product_id`, `quantity`, `price`

- **Dimension Tables:**
    1. `customers`
        - Columns: `customer_id`, `name`, `email`
    2. `products`
        - Columns: `product_id`, `name`, `category`
    3. `dates`
        - Columns: `date_id`, `date`, `weekday`

**b. Fact vs. Dimension Table Contents**

- **Fact Tables** store measurable, transactional data—events or numeric values (e.g., sales, quantities, amounts). Each row represents an event/transaction and typically contains foreign keys to dimension tables plus metrics.
- **Dimension Tables** store descriptive/contextual information (attributes) about the entities involved in facts (e.g., product details, customer info, dates). These provide the “who, what, when, where” context for analyzing facts.

---

## 4. Normalization & Denormalization Practice
- **a.** Normalize the following customer table by splitting out the address fields into a separate table. Show your resulting table schemas.
    ```
    customers (customer_id, name, street, city, state, zip)
    ```
- **b.** Write an SQL statement to join the normalized `customers` and `addresses` tables to produce the original denormalized view.
- **c.** Explain one benefit and one drawback of denormalizing tables for analytics.



### Answer

### a. Normalize the customer table

 customers (customer_id, name)
 addresses (address_id, street, city, state, zip, customer_id)

### b. Join the normalized tables

SELECT c.customer_id, c.name, a.street, a.city, a.state, a.zip
FROM customers c
INNER JOIN addresses a ON c.customer_id = a.customer_id

### c. Explain one benefit and one drawback of denormalizing tables for analytics

Denormalizing tables can improve the performance of analytical queries by reducing the number of joins required. However, it can also lead to data redundancy and inconsistencies if not properly maintained.

---

## 5. Data Warehouse Concepts
- **a.** Briefly explain the difference between OLAP and OLTP, including an example use case for each.
- **b.** What is a *data mart*? When might you use one?
- **c.** Describe what *slowly changing dimensions* (SCDs) are and why they matter.



### Answer

#### a. OLAP vs OLTP

- **OLAP (Online Analytical Processing):**  
  - Designed for complex, read-heavy queries and aggregations on large volumes of data.  
  - Supports business intelligence, reporting, and analytics.  
  - *Example use case:* A retailer uses OLAP to analyze sales trends across regions and time periods.

- **OLTP (Online Transaction Processing):**  
  - Optimized for high-speed, write-heavy operations like inserts, updates, and deletes.  
  - Supports daily business operations (e.g., order entry, banking transactions).  
  - *Example use case:* An e-commerce website uses OLTP to process customer orders and inventory updates in real time.

#### b. Data Mart

- A **data mart** is a focused subset of a data warehouse, tailored for a specific business line, department, or team (e.g., finance, marketing).  
- *Use case:* When a marketing team needs fast, dedicated access to campaign and customer data without querying the entire enterprise warehouse.

#### c. Slowly Changing Dimensions (SCDs)

- **Slowly Changing Dimensions** refer to dimension tables where attribute values may change over time (e.g., a customer's address or job title).  
- They matter because tracking historical changes accurately is crucial for reliable analytics and reporting.  
- *Example:*  
  - **Type 1 SCD:** Overwrites old data (no history retained).  
  - **Type 2 SCD:** Adds new rows for each change, preserving history (enables time-based analysis).






## 6. dbt Practice
- **a.** Write a dbt model (SQL) that calculates total sales per product from a `sales` table.  
- **b.** Write the yaml snippet to add a `not_null` and `unique` test on the `product_id` column of your model.



### Answer

-- dbt model: total sales per product
```sql
SELECT
    product_id,
    SUM(amount) AS total_sales
FROM sales
GROUP BY product_id;```



-- dbt test: Ensure product_id column is never null and unique
```yaml

version: 2
models:
  - name: total_sales_by_product
    columns:
      - name: product_id
        tests:
          - not_null
          - unique

```


---
## 7. Best Practices
- **a.** List three best practices for designing data warehouse schemas.
- **b.** Why is it recommended to use surrogate keys as primary keys?



### Answer

- **a.** List three best practices for designing data warehouse schemas.
  1. Use surrogate keys (integer IDs) as primary keys in fact and dimension tables.
  2. Document schema, relationships, and data lineage to ensure clarity and maintainability.
  3. Partition large tables by date or key for improved performance and scalability.

- **b.** Why is it recommended to use surrogate keys as primary keys?
  Surrogate keys are recommended as primary keys because they are unique, immutable, and not tied to business logic. This ensures that the keys remain consistent and reliable as the data evolves, and it simplifies the handling of changes in the business environment.

---

## Challenge

**Ride-Sharing Analytics Warehouse**

You are tasked with designing the data warehouse schema for a ride-sharing app. The app tracks drivers, riders, trips, and payments.

- **a.** List the fact and dimension tables you would create. Specify keys and a few example columns for each.
- **b.** Draw a schema diagram (or describe in text) showing the relationships between these tables.
- **c.** Write SQL to join trips with drivers and riders to produce a report of all completed trips, including driver and rider names, trip duration, and payment amount.
- **d.** Suggest one way to handle Slowly Changing Dimensions (SCD) for driver details (e.g., when a driver updates their vehicle).

---

### Answer

#### a. Fact and Dimension Tables

- **Fact Table:**
    - `trips_fact`
        - `trip_id` (PK)
        - `driver_id` (FK)
        - `rider_id` (FK)
        - `start_time`
        - `end_time`
        - `distance`
        - `payment_id` (FK)

- **Dimension Tables:**
    - `drivers_dim`
        - `driver_id` (PK)
        - `name`
        - `car_info`
        - `license_number`
    - `riders_dim`
        - `rider_id` (PK)
        - `name`
        - `email`
    - `payments_dim`
        - `payment_id` (PK)
        - `trip_id` (FK)
        - `amount`
        - `payment_time`
        - `method`

#### b. Schema Diagram Description

- `trips_fact` (central fact) references:
    - `drivers_dim` via `driver_id`
    - `riders_dim` via `rider_id`
    - `payments_dim` via `payment_id`
- `payments_dim` references `trips_fact` via `trip_id`
- This forms a star schema with `trips_fact` at the center and dimensions surrounding it.

#### c. SQL Join for Completed Trip Report

```sql
SELECT
    t.trip_id,
    d.name AS driver_name,
    r.name AS rider_name,
    t.start_time,
    t.end_time,
    (EXTRACT(EPOCH FROM t.end_time) - EXTRACT(EPOCH FROM t.start_time))/60 AS duration_minutes,
    p.amount AS payment_amount
FROM
    trips t
JOIN
    drivers d ON t.driver_id = d.driver_id
JOIN
    riders r ON t.rider_id = r.rider_id
JOIN
    payments p ON t.trip_id = p.trip_id
WHERE
    t.end_time IS NOT NULL; -- Completed trips only
```

#### d. Handling Slowly Changing Dimensions (SCD) for Drivers

- **Type 2 SCD Approach:**  
    - Add `driver_sk` (surrogate PK), `driver_id` (business key), `car_info`, `valid_from`, `valid_to`, and `is_current` columns to `drivers_dim`.
    - On update, insert a new row with updated `car_info`, set previous row's `valid_to` and `is_current = FALSE`, and new row's `is_current = TRUE`.
    - This allows historical queries on driver details as of any point in time.
